In [1]:
import pandas as pd

import glob

from pathlib import Path
import csv
import pickle

import os
import requests
import json

import quandl

from dotenv import load_dotenv
load_dotenv()

True

In [2]:
def load_obj(path):
    with open(path, 'rb') as f:
        return pickle.load(f)
    
def save_obj(obj, path ):
    with open(path, 'wb') as f:
        pickle.dump(obj, f, pickle.HIGHEST_PROTOCOL)

In [3]:
pkl_path_1 = Path('../Resources/complete_data_0_10000.pkl')
pkl_path_2 = Path('../Resources/incomplete_data_0_10000.pkl')
complete_data  = load_obj(pkl_path_1)
incomplete_data= load_obj(pkl_path_2)

In [7]:
# #complete_data
# for key in complete_data.keys():
#         complete_data[key].anything  

In [9]:
# symbol_df = pd.read_csv(
#     Path('../Resources/symbol_all_list.csv'),
#     index_col=0
# )
# len(symbol_df)

In [12]:
# pkl_path_3 = Path('../FilesExportComplete/data_complete_10000_11000.pkl')
# complete_data3  = load_obj(pkl_path_3)

In [9]:
i = 0
for key in complete_data.keys():
    i += 1
data_length = i
i = 0
print(data_length)

2519


In [15]:
#complete_data

In [10]:
key_list = []
for key in complete_data.keys():
    key_list.append(key)    

In [12]:
len(key_list)

2519

In [14]:
index = key_list[10]
index

'AAPL'

In [16]:
complete_data[index].head()

,close,volume,change,changePercent,changeOverTime,QUANTITY_FAILS,TotalShortVolume,TotalVolume,SHORToverTOTALvolume
Date,,,,,,,,,
2021-01-04,129.41,143301887,-3.28,-0.0247,-0.040483,5930.0,35765540.0,63704764.0,56.142646
2021-01-05,131.01,97664898,1.6,0.0124,-0.02862,95768.0,27372650.0,47733385.0,57.344875
2021-01-06,126.6,155087970,-4.41,-0.0337,-0.061318,0.0,42484021.0,73034054.0,58.170153
2021-01-07,130.92,109578157,4.32,0.0341,-0.029287,272530.0,30246166.0,51859535.0,58.323250
2021-01-08,132.05,105158245,1.13,0.0086,-0.020909,1750.0,29142533.0,50045713.0,58.231827


In [17]:
## Sum QUANTITY_FAILS
test_df = complete_data[index].copy()
test_df.head()

,close,volume,change,changePercent,changeOverTime,QUANTITY_FAILS,TotalShortVolume,TotalVolume,SHORToverTOTALvolume
Date,,,,,,,,,
2021-01-04,129.41,143301887,-3.28,-0.0247,-0.040483,5930.0,35765540.0,63704764.0,56.142646
2021-01-05,131.01,97664898,1.6,0.0124,-0.02862,95768.0,27372650.0,47733385.0,57.344875
2021-01-06,126.6,155087970,-4.41,-0.0337,-0.061318,0.0,42484021.0,73034054.0,58.170153
2021-01-07,130.92,109578157,4.32,0.0341,-0.029287,272530.0,30246166.0,51859535.0,58.323250
2021-01-08,132.05,105158245,1.13,0.0086,-0.020909,1750.0,29142533.0,50045713.0,58.231827


In [18]:
test_df.QUANTITY_FAILS.sum()

18371378.0

In [19]:
## Get sharesOutstanding and floatShares from Yahoo Finance 
## Please find a better source 
## IEX can be used for sharesOutstanding but not floatShares :\ 
import yfinance as yf

def get_ticker_info_yf(symbol):
    str_symbol = str(symbol)
    ticker = yf.Ticker(str_symbol)
    ticker_info = ticker.info
    try:
        sharesOutstanding = ticker_info['sharesOutstanding']
    except KeyError:
        sharesOutstanding = 0
    try:
        floatShares = ticker_info['floatShares']
    except KeyError:
        floatShares = 0
    dictionary = {
        'sharesOutstanding' : sharesOutstanding,
        'floatShares' : floatShares
    }
    return dictionary

In [36]:
test_list = key_list[1:11]
len(test_list)

10

In [40]:
# i = 0
# while i < 10:
#     print(test_list[i])
#     i += 1

In [59]:
test_info_dict = {}
infinity = float("Inf") ## Define Infinity value 
for i in test_list:
    df = complete_data[i].copy()
    ftd_sum = df.QUANTITY_FAILS.sum()
    i_info = get_ticker_info_yf(i)
    
    if i_info['sharesOutstanding'] != 0:
        pct_ftd_SO = ftd_sum / i_info['sharesOutstanding']
    else:
        pct_ftd_SO = 0
    if i_info['floatShares'] != 0:
        pct_ftd_float = ftd_sum / i_info['floatShares']
    else:
        pct_ftd_float = 0
    
## Above is better fix for dividing by infinity - checks before it happens 
#     pct_ftd_SO = ftd_sum / i_info['sharesOutstanding']
#     if pct_ftd_SO == infinity:
#         pct_ftd_SO = 0
#     pct_ftd_float = ftd_sum / i_info['floatShares'] 
#     if pct_ftd_float == infinity:
#         pct_ftd_float = 0    
        
    test_info_dict[i] = {
        'FTD_sum' : ftd_sum,
        'sharesOutstanding' : i_info['sharesOutstanding'],
        'floatShares' : i_info['floatShares'],
        'pct_ftd_SO' : pct_ftd_SO,
        'pct_ftd_float' : pct_ftd_float
    }
    
test_info_dict
## Works - issue with '0' values causing infite division though 

{'AA': {'FTD_sum': 948121.0,
  'sharesOutstanding': 187060000,
  'floatShares': 185713212,
  'pct_ftd_SO': 0.005068539506040842,
  'pct_ftd_float': 0.005105296439544646},
 'AAAU': {'FTD_sum': 8243234.0,
  'sharesOutstanding': 0,
  'floatShares': 0,
  'pct_ftd_SO': 0,
  'pct_ftd_float': 0},
 'AACG': {'FTD_sum': 3700237.0,
  'sharesOutstanding': 31376900,
  'floatShares': 8743140,
  'pct_ftd_SO': 0.11792869913853822,
  'pct_ftd_float': 0.4232160299389007},
 'AAIC': {'FTD_sum': 391773.0,
  'sharesOutstanding': 32331100,
  'floatShares': 29061104,
  'pct_ftd_SO': 0.012117527705521929,
  'pct_ftd_float': 0.013481008842609696},
 'AAL': {'FTD_sum': 47828675.0,
  'sharesOutstanding': 647513984,
  'floatShares': 642401246,
  'pct_ftd_SO': 0.07386508427901381,
  'pct_ftd_float': 0.07445296113264388},
 'AAN': {'FTD_sum': 117137.0,
  'sharesOutstanding': 31483200,
  'floatShares': 31096948,
  'pct_ftd_SO': 0.0037206192509020683,
  'pct_ftd_float': 0.0037668326808148506},
 'AAOI': {'FTD_sum': 44526

In [56]:
## Try to solve infinite division 
# AAAU
infinity = float("Inf")


i = 'AAAU'
df = complete_data[i].copy()
ftd_sum = df.QUANTITY_FAILS.sum()
i_info = get_ticker_info_yf(i)
pct_ftd_SO = ftd_sum / i_info['sharesOutstanding'] 
pct_ftd_float = ftd_sum / i_info['floatShares'] 
pct_ftd_float

C:\Users\watso\anaconda3\envs\pyvizenv\lib\site-packages\ipykernel_launcher.py:10: RuntimeWarning: divide by zero encountered in double_scalars
  # Remove the CWD from sys.path while we load stuff.
C:\Users\watso\anaconda3\envs\pyvizenv\lib\site-packages\ipykernel_launcher.py:11: RuntimeWarning: divide by zero encountered in double_scalars
  # This is added back by InteractiveShellApp.init_path()


inf

In [57]:
if pct_ftd_float == infinity:
    pct_ftd_float = 0
pct_ftd_float

0

In [55]:
infinity = float("Inf")
print(infinity)

inf

inf


inf